In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer
from sklearn.linear_model import Ridge

In [2]:
# Set parameters for linear regression

# ridge parameters for random search
fit_intercept = [True, False]
copy_X = [True, False]
alpha = [0.1, 0.5, 1, 2, 5, 10, 20, 50, 100]
tol = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
solver = ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']

# Create the random grid
random_grid = {'fit_intercept': fit_intercept,
                'copy_X': copy_X,
                'alpha': alpha,
                'tol': tol,
                'solver': solver}


In [3]:
#Define MASE Metric
def mean_absolute_scaled_error(y_true, y_pred):
    mase=0
    # Define numerator as the forecast error
    numerator = (np.abs(y_true - y_pred))

    # Define denominator as the mean absolute error of the in-sample one-step naive forecast
    y_true_ohne_1 = y_true[1:].reset_index(drop=True)
    y_true_ohne_ende = y_true[:-1].reset_index(drop=True)
    denominator = np.mean(np.abs(y_true_ohne_1 - y_true_ohne_ende))

    mase = np.mean(np.abs(numerator / denominator))

    return mase

scorer_mase= make_scorer(mean_absolute_scaled_error, greater_is_better=False)


In [4]:
# Load the data
data = pd.read_parquet('/Users/paddy/Documents/GitHub/Masterthesis_ML/data/03_30min_dataset.parquet')

# Convert the date column to datetime
data['date'] = pd.to_datetime(data['date']) #,format='%d/%m/%y %H:%M:%S').dt.strftime('%Y-%m-%d %H:%M:%S') 


In [5]:
#Feature engineering
# Create a new column for the time
data['time'] = [x for x in range(0, len(data))]

data['hour_of_day'] = data['date'].dt.hour
data['day_of_week'] = data['date'].dt.dayofweek
data['day_of_month'] = data['date'].dt.day
data['month_of_year'] = data['date'].dt.month
data['year'] = data['date'].dt.year

# make a weekend column
data['weekend'] = 0
data.loc[data['day_of_week'] == 5, 'weekend'] = 1
data.loc[data['day_of_week'] == 6, 'weekend'] = 1

#make a monday column
data['monday'] = 0
data.loc[data['day_of_week'] == 0, 'monday'] = 1

#make a tuesday column
data['tuesday'] = 0
data.loc[data['day_of_week'] == 1, 'tuesday'] = 1

#make a wednesday column
data['wednesday'] = 0
data.loc[data['day_of_week'] == 2, 'wednesday'] = 1

#make a thursday column
data['thursday'] = 0
data.loc[data['day_of_week'] == 3, 'thursday'] = 1

#make a friday column
data['friday'] = 0
data.loc[data['day_of_week'] == 4, 'friday'] = 1

#make a saturday column
data['saturday'] = 0
data.loc[data['day_of_week'] == 5, 'saturday'] = 1

#make a sunday column
data['sunday'] = 0
data.loc[data['day_of_week'] == 6, 'sunday'] = 1


# Drop the first three rows
data = data.dropna().reset_index(drop=True)

# Define the feature columns and the target column
feature_cols = ['time', 'hour_of_day', 'day_of_week', 'day_of_month', 'month_of_year', 'year', 'weekend', 'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
target_col = 'y'

# Drop nan values
data = data.dropna()

# Rename column count to y
data = data.rename(columns={'count': 'y'})

In [6]:
# Set y to the last column
cols = list(data.columns.values) #Make a list of all of the columns in the df
cols.pop(cols.index('y')) #Remove y from list
data = data[cols+['y']] #Create new dataframe with columns in the order you want

# drop the date column
train_data = np.delete(data, 0, 1) 

# Split the data into X and y
X_train, y_train = train_data[:, :-1], train_data[:, -1]


In [7]:
#vgl: https://lightrun.com/answers/scikit-learn-scikit-learn-grid_search-feeding-parameters-to-scorer-functions

# X and y to pandas dataframe
X_train = pd.DataFrame(X_train)
y_train = pd.Series(y_train)


In [8]:
# Set Cross Validation to 5 iterations
tscv = TimeSeriesSplit(n_splits=5)

In [9]:
# Instantiate model
model = Ridge()

search = RandomizedSearchCV(estimator = model, 
                               param_distributions = random_grid, 
                               n_iter = 25, 
                               cv = tscv,
                               refit=True, 
                               verbose=3, 
                               random_state=42, 
                               n_jobs = -1, 
                               scoring=scorer_mase, #make_scorer(scorer_mase, greater_is_better=True), #'neg_root_mean_squared_error', #
                               error_score=np.nan)


In [10]:
# Fit the random search model
search.fit(X_train, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits


[CV 3/5] END alpha=10, copy_X=False, fit_intercept=True, solver=sparse_cg, tol=0.001;, score=-1.196 total time=   0.0s
[CV 2/5] END alpha=0.5, copy_X=True, fit_intercept=True, solver=cholesky, tol=0.01;, score=-1.338 total time=   0.0s
[CV 4/5] END alpha=0.5, copy_X=True, fit_intercept=True, solver=cholesky, tol=0.01;, score=-2.436 total time=   0.0s
[CV 3/5] END alpha=0.5, copy_X=True, fit_intercept=True, solver=cholesky, tol=0.01;, score=-1.195 total time=   0.0s
[CV 1/5] END alpha=10, copy_X=False, fit_intercept=True, solver=sparse_cg, tol=0.001;, score=-1.196 total time=   0.0s
[CV 4/5] END alpha=10, copy_X=False, fit_intercept=True, solver=sparse_cg, tol=0.001;, score=-1.149 total time=   0.0s
[CV 5/5] END alpha=0.5, copy_X=True, fit_intercept=True, solver=cholesky, tol=0.01;, score=-1.066 total time=   0.0s
[CV 1/5] END alpha=100, copy_X=False, fit_intercept=False, solver=svd, tol=0.1;, score=-1.197 total time=   0.0s
[CV 2/5] END alpha=10, copy_X=False, fit_intercept=True, solve

/Users/paddy/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV 4/5] END alpha=5, copy_X=True, fit_intercept=False, solver=sag, tol=10;, score=-1.485 total time=   0.0s
[CV 1/5] END alpha=100, copy_X=False, fit_intercept=True, solver=sag, tol=0.0001;, score=-1.200 total time=   0.3s
[CV 1/5] END alpha=20, copy_X=True, fit_intercept=False, solver=saga, tol=10;, score=-1.532 total time=   0.0s
[CV 5/5] END alpha=5, copy_X=True, fit_intercept=False, solver=sag, tol=10;, score=-1.384 total time=   0.0s
[CV 3/5] END alpha=20, copy_X=True, fit_intercept=False, solver=saga, tol=10;, score=-2.021 total time=   0.0s
[CV 2/5] END alpha=20, copy_X=True, fit_intercept=False, solver=saga, tol=10;, score=-1.938 total time=   0.0s
[CV 4/5] END alpha=20, copy_X=True, fit_intercept=False, solver=saga, tol=10;, score=-1.444 total time=   0.0s
[CV 5/5] END alpha=20, copy_X=True, fit_intercept=False, solver=saga, tol=10;, score=-1.362 total time=   0.0s
[CV 1/5] END alpha=1, copy_X=False, fit_intercept=False, solver=lsqr, tol=0.001;, score=-1.208 total time=   0.0

/Users/paddy/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV 2/5] END alpha=0.5, copy_X=True, fit_intercept=True, solver=svd, tol=0.0001;, score=-1.338 total time=   0.0s
[CV 3/5] END alpha=0.5, copy_X=True, fit_intercept=True, solver=svd, tol=0.0001;, score=-1.195 total time=   0.0s
[CV 4/5] END alpha=0.5, copy_X=True, fit_intercept=True, solver=svd, tol=0.0001;, score=-2.436 total time=   0.0s
[CV 5/5] END alpha=0.5, copy_X=True, fit_intercept=True, solver=svd, tol=0.0001;, score=-1.066 total time=   0.0s
[CV 1/5] END alpha=10, copy_X=False, fit_intercept=True, solver=sparse_cg, tol=100;, score=-1.463 total time=   0.0s
[CV 2/5] END alpha=10, copy_X=False, fit_intercept=True, solver=sparse_cg, tol=100;, score=-1.554 total time=   0.0s
[CV 3/5] END alpha=10, copy_X=False, fit_intercept=True, solver=sparse_cg, tol=100;, score=-1.476 total time=   0.0s
[CV 4/5] END alpha=10, copy_X=False, fit_intercept=True, solver=sparse_cg, tol=100;, score=-1.445 total time=   0.0s
[CV 5/5] END alpha=10, copy_X=False, fit_intercept=True, solver=sparse_cg, t

/Users/paddy/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/paddy/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV 4/5] END alpha=100, copy_X=False, fit_intercept=True, solver=sag, tol=0.0001;, score=-1.152 total time=   1.2s
[CV 5/5] END alpha=0.5, copy_X=True, fit_intercept=True, solver=saga, tol=0.001;, score=-1.105 total time=   0.3s
[CV 5/5] END alpha=100, copy_X=False, fit_intercept=True, solver=sag, tol=0.0001;, score=-1.076 total time=   1.4s


/Users/paddy/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


RandomizedSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
                   estimator=Ridge(), n_iter=25, n_jobs=-1,
                   param_distributions={'alpha': [0.1, 0.5, 1, 2, 5, 10, 20, 50,
                                                  100],
                                        'copy_X': [True, False],
                                        'fit_intercept': [True, False],
                                        'solver': ['auto', 'svd', 'cholesky',
                                                   'lsqr', 'sparse_cg', 'sag',
                                                   'saga'],
                                        'tol': [0.0001, 0.001, 0.01, 0.1, 1, 10,
                                                100]},
                   random_state=42,
                   scoring=make_scorer(mean_absolute_scaled_error, greater_is_better=False),
                   verbose=3)

In [11]:
print('Best Score: %s' % search.best_score_)
print('Best Hyperparameters: %s' % search.best_params_)
print('Best Model: %s' % search.best_estimator_)
print('Best Index: %s' % search.best_index_)
print('CV Results: %s' % search.cv_results_)
print('Refit Time: %s' % search.refit_time_)
print('Scorer: %s' % search.scorer_)

Best Score: -1.1825137134613994
Best Hyperparameters: {'tol': 0.1, 'solver': 'svd', 'fit_intercept': False, 'copy_X': False, 'alpha': 100}
Best Model: Ridge(alpha=100, copy_X=False, fit_intercept=False, solver='svd', tol=0.1)
Best Index: 2
CV Results: {'mean_fit_time': array([0.01515703, 0.00927377, 0.01170921, 0.00718303, 0.00699377,
       0.00598803, 0.89765983, 0.00956693, 0.00772138, 0.01145506,
       0.00619588, 0.00933266, 0.00597091, 0.00654888, 0.00805073,
       0.24725032, 0.01016512, 0.01260395, 0.00688014, 0.00984368,
       0.01155901, 0.0068964 , 0.00918245, 0.00785742, 0.0064302 ]), 'std_fit_time': array([0.00712067, 0.0029089 , 0.00364689, 0.00238278, 0.00291762,
       0.00205681, 0.36177174, 0.00443576, 0.00426863, 0.0090459 ,
       0.00243387, 0.0036893 , 0.00220981, 0.00253577, 0.00250206,
       0.15367655, 0.00437668, 0.00438634, 0.00216336, 0.00187809,
       0.00746599, 0.00410833, 0.00398902, 0.00304306, 0.00144173]), 'mean_score_time': array([0.00558963, 0.